In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
import os
import json

# --- SECCIÓN 1: Carga de Datos y Generación/Carga de Reglas de Asociación ---

RULES_FILE = 'rules_recomendacion.json'
DATA_FILE = 'atelier-dataset-2023-2025-transacciones-unificado.csv'
df_completo = None

In [6]:
try:
    if os.path.exists(RULES_FILE):
        rules_df = pd.read_json(RULES_FILE)
        # Convertimos las columnas de listas (del JSON) a frozensets
        rules_df['antecedents'] = rules_df['antecedents'].apply(frozenset)
        rules_df['consequents'] = rules_df['consequents'].apply(frozenset)
        print(f"\nReglas de asociación cargadas desde '{RULES_FILE}'.")
    else:
        raise FileNotFoundError
except FileNotFoundError:
    print(f"\nNo se encontró el archivo de reglas '{RULES_FILE}'. Generando nuevas reglas...")
    # Si no existe el archivo de reglas, lo generamos desde cero
    try:
        df_completo = pd.read_csv(DATA_FILE)
        print(f"DataFrame '{DATA_FILE}' cargado exitosamente para la generación de reglas.")
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo de datos '{DATA_FILE}'. Usando datos simulados para demostración.")
        # Datos simulados para demostración
        data_simulada = {
            'transaccion_id': [f'T{i}' for i in range(1, 201)],
            'producto_talla': np.random.choice(['S', 'M', 'L', 'Talla Única'], size=200),
            'producto_color': np.random.choice(['Rojo', 'Negro', 'Azul', 'Beige', 'Verde', 'Blanco'], size=200),
            'producto_temporada': np.random.choice(['Verano', 'Invierno', 'Primavera', 'Todo el Año'], size=200),
            'vestido_estilo': np.random.choice(['Casual', 'Noche', 'Cóctel', 'Bohemio'], size=200),
            'vestido_condicion': np.random.choice(['Nuevo', 'Usado'], size=200),
            'producto_nombre': ['Vestido ' + str(i) for i in range(1, 201)],
            'reseña_rating': np.random.uniform(3.5, 5.0, size=200).round(1),
            'vestido_precio_venta': np.random.randint(500, 2500, size=200)
        }
        df_completo = pd.DataFrame(data_simulada)
        df_completo.loc[0:20, ['producto_talla', 'vestido_estilo']] = ['M', 'Casual']
        df_completo.loc[0:20, 'producto_temporada'] = 'Verano'

    # 2. Preprocesamiento de datos y generación del 'basket'
    atributos = ['producto_talla', 'producto_color', 'producto_temporada', 'vestido_estilo', 'vestido_condicion']
    df_completo.dropna(subset=['transaccion_id'] + atributos, inplace=True)
    df_completo.reset_index(drop=True, inplace=True)
    df_analisis = df_completo[['transaccion_id'] + atributos].copy()
    list_of_attributes = [df_analisis[attr].apply(lambda x: f"{attr}_{x}") for attr in atributos]
    items = pd.concat(list_of_attributes, ignore_index=True)
    df_basket = pd.DataFrame({
        'transaccion_id': df_analisis['transaccion_id'].repeat(len(atributos)).reset_index(drop=True),
        'item': items
    })
    basket = (df_basket.groupby(['transaccion_id', 'item'])['item'].count().unstack(fill_value=0).reset_index().set_index('transaccion_id'))
    basket = basket.applymap(lambda x: 1 if x > 0 else 0)

    # 3. Filtrar y aplicar Apriori para generar reglas
    min_transactions = 5 
    product_counts = basket.sum(axis=0)
    products_to_keep = product_counts[product_counts >= min_transactions].index
    basket_filtered = basket[products_to_keep]
    basket_bool = basket_filtered.astype(bool)

    try:
        frequent_itemsets = apriori(basket_bool, min_support=0.01, use_colnames=True)
        rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
        rules_df = rules_df.sort_values(['lift', 'confidence'], ascending=[False, False])
        print(f"\nSe generaron {len(rules_df)} reglas de asociación.")
        # Preparamos el DataFrame para guardarlo en JSON
        # Convertimos frozensets a listas para que puedan serializarse
        rules_df['antecedents'] = rules_df['antecedents'].apply(list)
        rules_df['consequents'] = rules_df['consequents'].apply(list)
        rules_df.to_json(RULES_FILE, orient='records', indent=4)
        print(f"Reglas guardadas en '{RULES_FILE}'.")
    except Exception as e:
        print(f"Error al generar reglas: {e}. Usando un DataFrame de reglas vacío.")
        rules_df = pd.DataFrame()



Reglas de asociación cargadas desde 'rules_recomendacion.json'.


In [7]:
def recomendar_por_similitud(atributos_elegidos, rules_df, df_transacciones_para_fallback, num_recomendaciones=2):
    """
    Busca la regla más similar a los atributos dados, priorizando lift y confianza.
    """
    if not rules_df.empty:
        mejores_reglas = []
        for _, row in rules_df.iterrows():
            antecedent = row['antecedents']
            coincidencias = len(antecedent.intersection(atributos_elegidos))
            if coincidencias > 0:
                mejores_reglas.append({
                    'regla': row,
                    'coincidencias': coincidencias
                })
        
        if mejores_reglas:
            mejores_reglas.sort(key=lambda x: (x['coincidencias'], x['regla']['lift'], x['regla']['confidence']), reverse=True)
            mejor_regla = mejores_reglas[0]['regla']
            recomendaciones = list(mejor_regla['consequents'])
            atributos_finales = [item for item in recomendaciones if item not in atributos_elegidos]
            
            print("\n--- Diagnóstico: Regla más similar encontrada ---")
            print(f"Antecedente: {set(mejor_regla['antecedents'])}")
            print(f"Consecuente: {set(mejor_regla['consequents'])}")
            print(f"Lift: {mejor_regla['lift']:.2f}, Confianza: {mejor_regla['confidence']:.2f}")

            return atributos_finales[:num_recomendaciones], "regla_mas_similar"

    print("💡 No se encontraron reglas de asociación. Usando el fallback de atributos populares.")
    columnas_atributos = ['producto_talla', 'producto_color', 'producto_temporada', 'vestido_estilo', 'vestido_condicion']
    list_of_attributes = [df_transacciones_para_fallback[attr].apply(lambda x: f"{attr}_{x}") 
                          for attr in columnas_atributos if attr in df_transacciones_para_fallback.columns and not df_transacciones_para_fallback[attr].empty]
    if not list_of_attributes:
        return [], "no_hay_datos"
    atributos_populares_serie = pd.concat(list_of_attributes)
    atributos_populares = (atributos_populares_serie.value_counts().head(num_recomendaciones + len(atributos_elegidos)).index.tolist())
    general_popular_attributes = [attr for attr in atributos_populares if attr not in atributos_elegidos]
    return general_popular_attributes[:num_recomendaciones], "atributos_populares"


In [8]:
# --- SECCIÓN 3: PRUEBA CON TU TRANSACCIÓN ESPECÍFICA ---
# Cargamos el DataFrame completo para la función de búsqueda de vestidos y fallback
if df_completo is None:
    try:
        df_completo = pd.read_csv(DATA_FILE)
    except FileNotFoundError:
        print(f"\nAdvertencia: El archivo de datos '{DATA_FILE}' no se encontró para la búsqueda final. Usando datos simulados.")
        df_completo = pd.DataFrame(data_simulada)



Advertencia: El archivo de datos 'atelier-dataset-2023-2025-transacciones-unificado.csv' no se encontró para la búsqueda final. Usando datos simulados.


NameError: name 'data_simulada' is not defined

In [ ]:
try:
    frequent_itemsets = apriori(basket_bool, min_support=0.01, use_colnames=True)
    rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
    rules_df = rules_df.sort_values(['lift', 'confidence'], ascending=[False, False])
    rules_df['antecedents'] = rules_df['antecedents'].apply(list)
    rules_df['consequents'] = rules_df['consequents'].apply(list)
    rules_df.to_json(RULES_FILE, orient='records', indent=4)
    print(f"Reglas guardadas en '{RULES_FILE}'.")
except Exception as e:
    print(f"Error al generar reglas: {e}")
    rules_df = pd.DataFrame()

In [ ]:
def recomendar_por_similitud(atributos_elegidos, rules_df, df_transacciones_para_fallback, num_recomendaciones=2):
    if not rules_df.empty:
        mejores_reglas = []
        for _, row in rules_df.iterrows():
            antecedent = frozenset(row['antecedents'])
            coincidencias = len(antecedent.intersection(atributos_elegidos))
            if coincidencias > 0:
                mejores_reglas.append({
                    'regla': row,
                    'coincidencias': coincidencias
                })
            if mejores_reglas:
                mejores_reglas.sort(key=lambda x: (x['coincidencias'], x['regla']['lift'], x['regla']['confidence']), reverse=True)
                mejor_regla = mejores_reglas[0]['regla']
                recomendaciones = [item for item in mejor_regla['consequents'] if item not in atributos_elegidos]
                print("\n--- Regla más similar encontrada ---")
                print(f"Antecedente: {mejor_regla['antecedents']}")
                print(f"Consecuente: {mejor_regla['consequents']}")
                return recomendaciones[:num_recomendaciones], "regla_mas_similar"
            print("⚠️ No se encontraron reglas relevantes. Usando atributos populares.")
            columnas_atributos = ['producto_talla', 'producto_color', 'producto_temporada', 'vestido_estilo', 'vestido_condicion']
            list_of_attributes = [
                df_transacciones_para_fallback[attr].apply(lambda x: f"{attr}_{x}")
                for attr in columnas_atributos if attr in df_transacciones_para_fallback.columns
          ]
            if not list_of_attributes:
                return [], "no_hay_datos"
            atributos_populares = pd.concat(list_of_attributes).value_counts().index.tolist()
            return [a for a in atributos_populares if a not in atributos_elegidos][:num_recomendaciones], "atributos_populares"


In [ ]:
def encontrar_vestidos_por_atributos(df_original, atributos_recomendados, excluidos=None):
    if not atributos_recomendados: return pd.DataFrame()
    filtros = []
    columnas_mapeo = {
        'producto_talla': 'producto_talla',
        'producto_color': 'producto_color',
        'producto_temporada': 'producto_temporada',
        'vestido_estilo': 'vestido_estilo',
        'vestido_condicion': 'vestido_condicion'
    }
    for attr_str in atributos_recomendados:
        for prefix, col_name in columnas_mapeo.items():
            if attr_str.startswith(f"{prefix}_"):
                valor = attr_str.replace(f"{prefix}_", "", 1)
                filtros.append(df_original[col_name] == valor)
                break
    if not filtros: return pd.DataFrame()
    filtro_combinado = filtros[0]
    for f in filtros[1:]:
        filtro_combinado &= f
    resultados = df_original[filtro_combinado].copy()
    if excluidos:
        resultados = resultados[~resultados['producto_nombre'].isin(excluidos)]
    return resultados.head(5) 

In [ ]:
if df_completo is None:
    try:
        df_completo = pd.read_csv(DATA_FILE)
    except FileNotFoundError:
        print(f"\nAdvertencia: El archivo de datos '{DATA_FILE}' no se encontró para la búsqueda final. Usando datos simulados.")
        df_completo = pd.DataFrame(data_simulada)


print("\n" + "="*50)
print("--- PRUEBA DEL SISTEMA CON LA TRANSACCIÓN QUE ME DISTE ---")

datos_transaccion = {
    'producto_nombre': 'Vestido de Noche Meirius Largo Verde con Hombro Descubierto',
    'producto_talla': 'XL',
    'producto_color': 'verde',
    'producto_temporada': 'Todo el Año',
    'vestido_estilo': 'De Noche',
    'vestido_condicion': 'Nuevo',
}

atributos_elegidos = frozenset([
    f"producto_talla_{datos_transaccion['producto_talla']}",
    f"producto_color_{datos_transaccion['producto_color']}",
    f"producto_temporada_{datos_transaccion['producto_temporada']}",
    f"vestido_estilo_{datos_transaccion['vestido_estilo']}",
    f"vestido_condicion_{datos_transaccion['vestido_condicion']}"
])

print(f"La transacción del cliente incluye un vestido con estos atributos:\n{list(atributos_elegidos)}")
atributos_recomendados, metodo = recomendar_por_similitud(
    atributos_elegidos=atributos_elegidos,
    rules_df=rules_df,
    df_transacciones_para_fallback=df_completo,
    num_recomendaciones=3
)

print(f"\nEl sistema recomienda buscar vestidos con los siguientes atributos:\n{atributos_recomendados}")
print(f"(Método utilizado: {metodo})")

vestidos_recomendados = encontrar_vestidos_por_atributos(
    df_original=df_completo,
    atributos_recomendados=atributos_recomendados,
    excluidos=[datos_transaccion['producto_nombre']]
)

if not vestidos_recomendados.empty:
    print("\n¡Vestidos recomendados para el cliente! (coinciden con los atributos sugeridos)")
    print("-------------------------------------------------------------------------")
    
    columnas_a_mostrar = [
        'producto_nombre',
        'reseña_rating',
        'producto_temporada',
        'producto_talla',
        'producto_color',
        'vestido_estilo',
        'vestido_condicion',
        'vestido_precio_venta'
    ]
    
    columnas_existentes = [col for col in columnas_a_mostrar if col in vestidos_recomendados.columns]
    
    df_final_recomendaciones = vestidos_recomendados[columnas_existentes]
    print(df_final_recomendaciones)
else:
    print("\nLo siento, no se encontraron vestidos en el catálogo que coincidan con los atributos recomendados.")

In [ ]:

# --- TRANSACCIÓN DE PRUEBA ---
datos_transaccion = {
    'producto_nombre': 'Vestido de Noche Meirius Largo Verde con Hombro Descubierto',
    'producto_talla': 'XL',
    'producto_color': 'verde',
    'producto_temporada': 'Todo el Año',
    'vestido_estilo': 'De Noche',
    'vestido_condicion': 'Nuevo',
}

atributos_elegidos = frozenset([
    f"producto_talla_{datos_transaccion['producto_talla']}",
    f"producto_color_{datos_transaccion['producto_color']}",
    f"producto_temporada_{datos_transaccion['producto_temporada']}",
    f"vestido_estilo_{datos_transaccion['vestido_estilo']}",
    f"vestido_condicion_{datos_transaccion['vestido_condicion']}"
])

print("\n" + "="*50)
print("🎯 Atributos del vestido elegido:")
print(list(atributos_elegidos))

atributos_recomendados, metodo = recomendar_por_similitud(
    atributos_elegidos=atributos_elegidos,
    rules_df=rules_df,
    df_transacciones_para_fallback=df_completo,
    num_recomendaciones=3
)

print(f"\n🔎 Atributos recomendados: {atributos_recomendados} (Método: {metodo})")

vestidos_recomendados = encontrar_vestidos_por_atributos(
    df_original=df_completo,
    atributos_recomendados=atributos_recomendados,
    excluidos=[datos_transaccion['producto_nombre']]
)

if not vestidos_recomendados.empty:
    print("\n✅ Vestidos recomendados:")
    columnas_a_mostrar = [
        'producto_id',
        'producto_nombre',
        'reseña_rating',
        'producto_temporada',
        'producto_talla',
        'producto_color',
        'vestido_estilo',
        'vestido_condicion',
        'vestido_precio_venta'
    ]
    columnas_existentes = [col for col in columnas_a_mostrar if col in vestidos_recomendados.columns]
    df_final_recomendaciones = vestidos_recomendados[columnas_existentes]
    print(df_final_recomendaciones)

    vestidos_json = df_final_recomendaciones.to_dict(orient='records')

    joblib.dump(rules_df, 'rules_model.pkl')
    joblib.dump(frequent_itemsets, 'frequent_itemsets.pkl')
    joblib.dump(basket_filtered, 'basket_filtered.pkl')
    print("✅ Modelos exportados como archivos .pkl")
    print("\n📦 Formato JSON:")
    print(json.dumps(vestidos_json, indent=4, ensure_ascii=False))
else:
    print("\n❌ No se encontraron vestidos recomendados con los atributos sugeridos.")
